In [ ]:
import pandas as pd

In [ ]:
# 1 & 2: Reading the CSV file --- EXTRACTION ---
df = pd.read_csv("lab1_data.csv", sep=";")
print(df)

In [ ]:
# 3: --- TRANSFORMATION ---
# Removing white spaces 
dirty_df = df
dirty_df[["id", "name", "price", "currency", "created_at"]] = dirty_df[["id", "name", "price", "currency", "created_at"]].apply(lambda x: x.str.strip())
dirty_df 

In [ ]:
# Fixing format for dates "created_at" 
dirty_df["created_at"] = pd.to_datetime(dirty_df["created_at"], errors='coerce')
dirty_df

In [ ]:
# Datatype conversion, from INTEGER ---> FLOAT
# index 13: string "free" ---> 0.0 Not an issue here because the intend is clear, by writing "free", the product is expected to have price zero
# index 51: string "not_available"  ---> Nan
dirty_df["price"] = pd.to_numeric(dirty_df["price"], errors="coerce")
dirty_df


In [ ]:
# 4: Flagging problems like missing data, prices < 0 or > 5000
luxury = 5000
checked_columns = ["id", "name", "price", "currency", "created_at"]
dirty_df["is_flagged"] = dirty_df[checked_columns].isna().any(axis=1)|(dirty_df["price"] < 0)|(dirty_df["price"] > luxury)
dirty_df

In [ ]:
# 5: Reject impossible values
# Rejected: missing id, missing price, price < 0, missing currency
rejected_condition = ( 
    (dirty_df["id"].isna()) | 
    (dirty_df["price"] < 0) | 
    (dirty_df)["price"].isna() |
    ( dirty_df["currency"].isna()) 
    )

rejected_df = dirty_df[rejected_condition].copy()
accepted_df = dirty_df[~rejected_condition].copy()
rejected_df

# pdf 6 sida 52 reject när obligatoriskt fält saknas = currency 
# pris at 0.0 is not an issue as string was "free" and intend is clear

In [ ]:
# 6c: --- LOAD ---
# Rejected_products CSV file
# Use of commas instead of semicolon as not specified
rejected_df.to_csv("rejected_products.csv", index=False)
print("rejected_products created!")

In [ ]:
# Prepared if neded but not required as per instructions
# accepted_df.to_csv("accepted_products",index=False)


In [ ]:
# 6a: analytics_summary preparation with utility methods
# From accepted_df
analytics = {
    "snitpris" : [accepted_df['price'].mean().round(2)],
    "medianpris" : [accepted_df['price'].median()],
    "antal_produkter" : [len(accepted_df)],
    "antal_produkter_inget_pris" : [rejected_df['price'].isna().sum()],
    }
analytics

In [ ]:
# Conversion From dict ---> DataFrame ---> CSV file
analytics_summary = pd.DataFrame(analytics)
analytics_summary.to_csv("analytics_summary.csv", index=False)
print("analytics_summary created!")

In [ ]:
# 6b: price_analysis.csv BONUS
# Top 10 most expensive products / top 10 dyraste produkter
top_10_expensive = pd.DataFrame(accepted_df).sort_values(by=["price"], ascending=False).head(10).copy()
top_10_expensive


In [ ]:
# Top 10 (products) price outliers / Top 10 produkter med mest avvikande pris 
# After speaking with Kristoffer => 2 outliers versions: including flagged outliers rows and excluding flagged outliers rows

# --- outliers INCLUDING flagged rows ---
outliers_all = accepted_df.copy()
median_price_all = outliers_all["price"].median()
outliers_all["deviation"] = (outliers_all["price"] - median_price_all).abs()
#outliers_all

top_10_outliers_all  =( outliers_all.sort_values(by="deviation", ascending=False).head(10).copy() )
top_10_outliers_all

In [ ]:
# --- outliers EXCLUDING flagged rows ---
outliers_not_flagged_df = accepted_df[accepted_df["is_flagged"] == False].copy()
median_price_not_flagged = outliers_not_flagged_df["price"].median()
outliers_not_flagged_df["deviation"]=(outliers_not_flagged_df["price"] - median_price_not_flagged).abs()

top_10_outliers_not_flagged = ( outliers_not_flagged_df.sort_values(by="deviation", ascending=False).head(10).copy() )
top_10_outliers_not_flagged

In [ ]:
# Merging the 3 dataframes, top_10_expensive & the 2 outliers
top_10_expensive["category"] = "top_10_expensive"
top_10_outliers_all["category"] = "outliers_including_flagged"
top_10_outliers_not_flagged["category"] = "outliers_excluding_flagged"

price_analyis = pd.concat([top_10_expensive, top_10_outliers_all, top_10_outliers_not_flagged], ignore_index=True)
price_analyis

In [19]:
price_analyis.to_csv("price_analysis_version1.csv", index=False)
print("price_analysis_version1 created!")

price_analysis_version1 created!


### <center>ANOTHER WAY FOR 6b price_analysis.csv BONUS</center> 


In [20]:
# Top 10 most expensive
top_10_expensive_3 = (
    accepted_df
    .sort_values("price", ascending=False)
    .head(10)
    .copy()
    .assign(category="Top 10 expensive")
)

# Top 10 lowest (trick to get the most "negative" outliers)
top_low_3 = (
    accepted_df
    .sort_values("price", ascending=True)   
    .head(10)
    .copy()
    .assign(category="Top 10 cheap")
)

price_analysis3 = pd.concat([top_10_expensive_3, top_low_3], ignore_index=True)
price_analysis3

,id,name,price,currency,created_at,is_flagged,category
0,SKU-1039,coat,999999.0,SEK,2024-10-15,True,Top 10 expensive
1,SKU-1021,ring,25000.0,SEK,2024-06-05,True,Top 10 expensive
2,SKU-1020,watch,15000.0,SEK,2024-06-01,True,Top 10 expensive
3,SKU-1032,suit,5000.0,SEK,2024-09-01,False,Top 10 expensive
4,SKU-1040,parka,4500.0,SEK,2024-11-01,False,Top 10 expensive
5,SKU-1027,winter jacket,3500.0,SEK,2024-07-15,False,Top 10 expensive
6,SKU-1016,coat,2500.0,SEK,2024-05-01,False,Top 10 expensive
7,SKU-1034,blazer,2200.0,SEK,2024-09-10,False,Top 10 expensive
8,SKU-1010,boots,1500.0,SEK,2024-02-15,False,Top 10 expensive
9,SKU-1047,heels,1500.0,SEK,2024-12-15,False,Top 10 expensive


In [21]:
price_analysis3.to_csv("price_analysis_version2.csv", index=False)
print("price_analysis_version2 created!")

price_analysis_version2 created!
